[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

# Fraud detection using kNN

In [1]:
import requests
import io
import os

try:
    os.mkdir("./tmp")
except:
    pass

external_files = [
  {
      "url":"https://raw.githubusercontent.com/haalcala/NYP-SDAI/master/ITI105/project/ds_util.py",
      "local_file":"ds_util.py"
  },
  {
      "url":"https://raw.githubusercontent.com/haalcala/NYP-SDAI/master/ITI105/project/files/bs140513_032310.csv",
      "local_file":"bs140513_032310.csv"
  },
]

for ext_file in external_files:
  req = requests.get(ext_file["url"])

  print(ext_file["local_file"], len(req.content))

  f = open("./tmp/" + ext_file["local_file"],"wb")

  f.write(req.content)

ds_util.py 1917
bs140513_032310.csv 48391391



# load and prepare data


In [2]:
import tmp.ds_util as util

ds_util = util.DSUtil()
ds_util.load_csv("./tmp/bs140513_032310.csv")

# analyse data composition

ds_util.blow_my_mind()

args: {}
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594643 entries, 0 to 594642
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   step         594643 non-null  int64  
 1   customer     594643 non-null  object 
 2   age          594643 non-null  object 
 3   gender       594643 non-null  object 
 4   zipcodeOri   594643 non-null  object 
 5   merchant     594643 non-null  object 
 6   zipMerchant  594643 non-null  object 
 7   category     594643 non-null  object 
 8   amount       594643 non-null  float64
 9   fraud        594643 non-null  int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 45.4+ MB
None
unique step    180
dtype: int64
unique customer    4112
dtype: int64
unique age    8
dtype: int64
unique gender    4
dtype: int64
unique zipcodeOri    1
dtype: int64
unique merchant    50
dtype: int64
unique zipMerchant    1
dtype: int64
unique category    15
dtype: int64
unique amount    23767


## drop columns not needed for calculations and get dummy variables

In [3]:
import pandas as pd
import numpy as np

df = pd.DataFrame(np.random.randn(1000,1000))

df[df > 0.9] = pd.np.nan

df

C:\Users\Harold\anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  


,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,-0.487958,0.181207,0.569298,NaN,-1.074873,-0.178312,-0.144497,-0.436534,0.333738,-1.863940,...,0.690407,NaN,NaN,-0.184662,0.029875,0.479809,-0.623100,-0.315957,-1.287159,-0.342695
1,NaN,0.233155,-0.214892,NaN,-0.455700,0.461350,NaN,0.439163,NaN,-1.895391,...,-0.512239,NaN,NaN,0.347063,-0.824702,-0.312220,-0.715589,NaN,-1.191969,0.532857
2,-0.072062,0.439689,-0.548018,NaN,NaN,-0.823063,NaN,NaN,-1.596739,NaN,...,-1.181731,0.316547,-1.049419,-0.900866,-0.171930,NaN,-1.966979,0.658311,-1.329613,0.106266
3,0.630204,-0.108908,-0.819370,-1.716553,-1.051517,0.135221,NaN,NaN,0.203364,-0.063989,...,-0.172502,0.421785,-0.848524,NaN,-0.497152,0.594932,-0.247179,-1.000141,NaN,-0.341003
4,0.430907,-0.468488,0.482602,-1.303430,-2.642456,NaN,-1.443369,0.569730,0.601341,-1.267899,...,0.422383,-0.444947,-0.435612,0.445913,0.083783,NaN,0.121147,-0.303536,NaN,-1.629545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,-0.458364,-0.721773,-0.829696,0.861534,NaN,-0.328444,0.333923,NaN,-0.377719,...,-0.349374,-0.469124,-0.025978,NaN,0.426916,0.261585,NaN,NaN,-0.636201,-1.310921
996,NaN,-1.157151,-0.495458,NaN,0.510314,-0.024344,-0.699735,0.720589,-1.750781,NaN,...,NaN,-1.605697,-0.565052,-0.781888,-0.477889,0.131242,-0.337421,NaN,-0.287449,-1.224635
997,0.859751,NaN,NaN,NaN,-0.270619,-0.999412,-0.461670,NaN,NaN,NaN,...,0.479466,0.888388,-1.176753,NaN,-0.547699,NaN,NaN,-0.598623,0.252435,-0.170652
998,0.702028,NaN,0.604036,-0.903400,NaN,-1.142338,-0.399235,-0.068417,-0.033800,0.679912,...,-0.254890,-0.497067,0.285923,-0.556788,NaN,-0.595366,-0.795054,NaN,0.648686,NaN


In [ ]:
ds_util.drop_columns(["customer", "zipMerchant", "zipcodeOri"])

df = ds_util.get_dummies()

df

# kNN Stuffs                                                     


## Training


In [ ]:
#We need to import the k-NN Classifier from skleart.neighbors
from sklearn.neighbors import KNeighborsClassifier

target_column = "fraud"

#Set the target column as our label (target value to be predicted)
YTrain = df[target_column]

#Remove the target column from our input variables
XTrain = df.drop(target_column, axis=1)

#Create a k-NN Classifier
classifier = KNeighborsClassifier(n_neighbors=1)

def train_fn():
    #Train the classifier using our training data
    classifier.fit(XTrain, YTrain)

ds_util.activity_wrapper("Training", train_fn)

## Testing


In [ ]:
XTest_fraud = df.loc[df.fraud==1].drop(target_column, axis=1).iloc[0].values
XTest_not_fraud = df.loc[df.fraud==0].drop(target_column, axis=1).iloc[0].values

# XTest = pd.DataFrame([
#     # this should give value 1 result
#     XTest_fraud,
#     # this should give value 0 result
#     XTest_not_fraud
# ])

# for index, row in XTest.iterrows():
#     print(classifier.predict(row.values.reshape(1, -1)))

total_test_count = 2
tests_count_done = 0
tests_passed = 0
tests_failed = 0

def test_fn():
    global tests_count_done, tests_passed, tests_failed
    
    tests_count_done = tests_count_done+1
    if classifier.predict(XTest_fraud.reshape(1,-1))[0] == 1:
        tests_passed = tests_passed+1
        print("Tested fraud entry to be fraud")
    else:
        tests_failed = tests_failed+1
        print("Failed testing fraud to be fraud")
    
    tests_count_done = tests_count_done+1
    if classifier.predict(XTest_not_fraud.reshape(1,-1))[0] == 0:
        tests_passed = tests_passed+1
        print("Tested not fraud entry to be not fraud")
    else:
        tests_failed = tests_failed+1
        print("Failed testing not fraud to be not fraud")
        
ds_util.activity_wrapper("Testing", test_fn)
     
print("Test result: {}/{} done. passed: {}, failed: {}.".format(tests_count_done, total_test_count, tests_passed, tests_failed))

print("All done.")